# Data

In [1]:
from load_data import load_data, Cols
df = load_data()
df[Cols.library].unique()

array(['Qcodes Community', 'Instrumentkit', 'Pymeasure', 'Qcodes', '',
       'Instrumental', 'Pytango'], dtype=object)

In [3]:
for cat in df[Cols.category].tolist():
    break

cat

['Power Supplies']

# Instrumentkit

In [48]:
import os
from scraping_utils import get_docs_from_link
import pandas as pd

if os.path.isfile("instrumentkit.csv"):
    df_inskit = pd.read_csv("instrumentkit.csv")
else:
    df_inskit = df[df[Cols.library] == "Instrumentkit"]
    if Cols.docstring not in df_inskit.columns:
        df_inskit[Cols.docstring] = df_inskit[Cols.docs].map(get_docs_from_link)
        df_inskit.to_csv("instrumentkit.csv")
        
df_inskit.head(2)


,Unnamed: 0,"Yearly revenue (millions, USD)",Vendor headquarters,Device Description,Device,Field 1,Vendor wikipedia or cruncbase description,Library,Vendor,Vendor logo,...,Device datasheet (PDF),Device picture,Device Description Link,GitHub link to Python driver (NOT LINK TO DOCS ON GITHUB),Python docs link,Device Price,ChatGPT code,Obsolete,Notes,docstring
0,4,550.0,USA,The PM100USB Power and Energy Meter Interface ...,PM 100 USB,606.0,"Thorlabs, Inc. is an American privately held o...",Instrumentkit,Thorlabs,"[OrderedDict([('id', 'attRmyBjaipm6atue'), ('w...",...,https://www.thorlabs.com/_sd.cfm?fileName=1957...,"[OrderedDict([('id', 'attE3gJZRoLQpO3nR'), ('w...",https://www.thorlabs.com/newgrouppage9.cfm?obj...,https://github.com/instrumentkit/InstrumentKit...,https://instrumentkit.readthedocs.io/en/latest...,487.99,NaN,NaN,NaN,\n\nSource code for instruments.thorlabs.pm100...
1,5,3500.0,"Irvine, California, United States",The model 8742 is a 4-axis open-loop intellige...,Picomotorcontroller 8742,384.0,Newport provides a wide range of photonics tec...,Instrumentkit,Newport,"[OrderedDict([('id', 'attXej1yb0ZjvVVtv'), ('w...",...,https://www.newport.com/medias/sys_master/imag...,"[OrderedDict([('id', 'attloDMGvxGciAsjA'), ('w...",https://www.newport.com/p/8742,https://github.com/instrumentkit/InstrumentKit...,https://instrumentkit.readthedocs.io/en/latest...,1260.00,NaN,NaN,NaN,\n\nSource code for instruments.newport.agilis...


In [79]:
!mkdir instrumentkit_snippets
import numpy as np
from tqdm.auto import tqdm
from query_chatgpt import query_chatgpt

for idx, row in tqdm(df_inskit.iterrows(), total=len(df_inskit)):
    library = row[Cols.library]
    device_name = row[Cols.device_name]
    corrected = row[Cols.correct_device_name]
    device = corrected if corrected is not np.nan else device_name
        
    # Don't redo one that already exists
    device_file = device.replace("/", "-")
    device_path = f"instrumentkit_snippets/{device_file}.txt"
    if os.path.isfile(device_path):
        continue
    
    category = row[Cols.category]
    # For some reason they are formatted as '"['text']"'
    category = eval(category)[0] if category is not np.nan else ""
    
    docstring = row[Cols.docstring]
    code = query_chatgpt(library, device, category, docstring)
    
    with open(device_path, "w") as f:
        f.write(code)
    

  0%|          | 0/67 [00:00<?, ?it/s]

# Qcodes

In [ ]:
!pip install -qqq qcodes

In [2]:
import os
import pandas as pd
from github_utils import get_raw_code_for_device

if os.path.isfile("qcodes.csv"):
    df_qcodes = pd.read_csv("qcodes.csv")
else:
    df_qcodes = df[df[Cols.library] == "Qcodes"]
    if Cols.docstring not in df_qcodes.columns:
        df_qcodes[Cols.docstring] = df_qcodes[Cols.github_link].map(get_raw_code_for_device)
        df_qcodes.to_csv("qcodes.csv")
        
df_qcodes[[Cols.device_name, Cols.correct_device_name, Cols.github_link, Cols.docstring]].head(2)


,Device,Corrected device name,GitHub link to Python driver (NOT LINK TO DOCS ON GITHUB),docstring
0,N 52 XX,KEYSIGHT N52XXA,NaN,NaN
1,Keysight 33511 B,KEYSIGHT 33511B,https://github.com/QCoDeS/Qcodes/blob/v0.25.0/...,from functools import partial\nimport logging\...


In [ ]:
from create_device_snippets import create_device_snippets

create_device_snippets(df_qcodes, "qcodes_snippets")

# PyTango

In [5]:
import os
import pandas as pd
from github_utils import get_raw_code_for_device

if os.path.isfile("pytango.csv"):
    df_tango = pd.read_csv("pytango.csv")
else:
    df_tango = df[df[Cols.library] == "Pytango"]
    if Cols.docstring not in df_tango.columns:
        df_tango[Cols.docstring] = df_tango[Cols.github_link].map(get_raw_code_for_device)
        df_tango.to_csv("pytango.csv")
        
print(f"{len(df_tango)} samples for Pytango")
df_tango[[Cols.device_name, Cols.correct_device_name, Cols.github_link, Cols.docstring]].head(2)


42 samples for Pytango


,Device,Corrected device name,GitHub link to Python driver (NOT LINK TO DOCS ON GITHUB),docstring
0,Keithleymultimeters,NaN,NaN,NaN
1,Gepressctrlpace 5000,Druck PACE5000,https://github.com/tiagocoutinho/gepace/blob/m...,import asyncio\nimport urllib.parse\n\nimport ...


In [24]:
from create_device_snippets import create_device_snippets

create_device_snippets(df_tango, "pytango_snippets")

  0%|          | 0/42 [00:00<?, ?it/s]

# Pymeasure

In [2]:
import os
import pandas as pd
from github_utils import get_raw_code_for_device

if os.path.isfile("pymeasure.csv"):
    df_measure = pd.read_csv("pymeasure.csv")
else:
    df_measure = df[df[Cols.library] == "Pymeasure"]
    if Cols.docstring not in df_measure.columns:
        df_measure[Cols.docstring] = df_measure[Cols.github_link].map(get_raw_code_for_device)
        df_measure.to_csv("pymeasure.csv")
        
df_measure[[Cols.device_name, Cols.correct_device_name, Cols.github_link, Cols.docstring]].head(2)


[NbConvertApp] Converting notebook /var/folders/2q/xy9rl8zn00b9_3ync0yk3jm40000gn/T/tmpttar0q_0/device.ipynb to script
[NbConvertApp] Writing 34276 bytes to /var/folders/2q/xy9rl8zn00b9_3ync0yk3jm40000gn/T/tmpttar0q_0/device.py
/var/folders/2q/xy9rl8zn00b9_3ync0yk3jm40000gn/T/ipykernel_23302/316810200.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_measure[Cols.docstring] = df_measure[Cols.github_link].map(get_raw_code_for_device)


,Device,Corrected device name,GitHub link to Python driver (NOT LINK TO DOCS ON GITHUB),docstring
6,Lakeshore 421,NaN,https://github.com/pymeasure/pymeasure/blob/ma...,#\n# This file is part of the PyMeasure packag...
8,SR 570,NaN,https://github.com/pymeasure/pymeasure/blob/ma...,#\n# This file is part of the PyMeasure packag...


In [5]:
from create_device_snippets import create_device_snippets

create_device_snippets(df_measure, "pymeasure_snippets")

  0%|          | 0/74 [00:00<?, ?it/s]

# Debugging decoding/counting tokens

In [27]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
e = enc.encode(code)

enc.decode(e[:15000])

'# -*- coding: utf-8 -*-\n\n########################## Copyrights and license ############################\n#                                                                            #\n# Copyright 2011-2015  Christian Lupien <christian.lupien@usherbrooke.ca>    #\n#                                                                            #\n# This file is part of pyHegel.  http://github.com/lupien/pyHegel            #\n#                                                                            #\n# pyHegel is free software: you can redistribute it and/or modify it under   #\n# the terms of the GNU Lesser General Public License as published by the     #\n# Free Software Foundation, either version 3 of the License, or (at your     #\n# option) any later version.                                                 #\n#                                                                            #\n# pyHegel is distributed in the hope that it will be useful, but WITHOUT     #\n# ANY WARRAN